In [1]:
pip install trino

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 2.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
from sys import path
path.append("/home/ec2-user/SageMaker/data-science-development/utils")
path.append("/home/ec2-user/SageMaker/data-science-development/config")

import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import pyarrow.parquet as pq
import seaborn as sns
import matplotlib.ticker as mtick

import itertools
import s3fs
import os
import torch
import random
import json
# import boto
import scipy
# import sktime
import trino

from torch import nn
from torch.autograd import Variable
from torch.utils.data import TensorDataset, DataLoader, WeightedRandomSampler

from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight

from trino.dbapi import connect
from trino.auth import OAuth2Authentication, BasicAuthentication

from datetime import datetime
# from config import Config  
from collections import defaultdict, Counter
from tqdm import tqdm 
from itertools import zip_longest
from contextlib import closing

tqdm.pandas()

In [3]:
conn = connect(
           host=f'trino.prd.rgn',
           port=443,
           user="roan.schellingerhout@randstadgroep.nl",
           catalog='iceberg',
           verify="certificate",
           http_scheme="https",
           auth=OAuth2Authentication()
     )

In [4]:
cur = conn.cursor()

In [6]:
request_data = cur.execute(open("./sql/action_log.sql").read()).fetchall()

Open the following URL in browser for the external authentication:
https://trino.prd.rgn/oauth2/token/initiate/e63b6f247caa065134545a85dde01e9657c394b1f3da9f1455f47b2c7ff2ed05


In [7]:
headers = [i[0] for i in cur.description]

headers

['candidate_id', 'request_mondriaan_number', 'action_code', 'date_action']

In [8]:
df = pd.DataFrame(data=request_data, columns=headers)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3850886 entries, 0 to 3850885
Data columns (total 4 columns):
 #   Column                    Dtype         
---  ------                    -----         
 0   candidate_id              int64         
 1   request_mondriaan_number  int64         
 2   action_code               object        
 3   date_action               datetime64[ns]
dtypes: datetime64[ns](1), int64(2), object(1)
memory usage: 117.5+ MB


In [11]:
df.to_csv("./cleaner_data/actions.csv")